### import

In [3]:
import pandas as pd
import numpy as np
import pickle
import os
from os.path import join
import re
from file_function import re_name_upload, write_log_csv_upload
from tqdm import tqdm
from datetime import datetime, timedelta

# dtype 정의
debt_dtype = {'채무자키':str, '타채무자키':str, '담당자키':str}
account_dtype = {'채무자키':str, '계좌키':str, '타채무자키':str}
grt_dtype = {'채무자키':str, '계좌키':str, '타채무자키':str, '보증인키':str}
rehabilitation_dtype = {'채무자키':str, '계좌키':str, '분납키':str, '사건키':str}
credit_dtype = {"채무자키":str, "계좌키":str, '보증인키':str, "계좌번호":str, "심의차수":str, "변제금수취계좌":str}
event_dtype = {'채무자키':str, '법조치키':str, '계좌키':str, '관련법조치키':str, '법취하키':str, '타법조치키':str, '타채무자키':str, '관할법원코드':str}
deposit_dtype = {'채무자키':str, '입금키':str, '계좌키':str, '계좌번호':str, '입금고정키':str, '타채무자키':str}

### 함수 및 전역변수

###### 불필요함수

In [ ]:
# wd = r"D:\0.전산\전산조회"
p_totalyOut = re.compile(r"환매|매각|종결")
p_debt = re.compile(r"파산")
p_grt = re.compile(r"파산\(면책\)|환매|매각|종결|보증면탈")
# p_debt = re.compile(r"개인회생\(면책\)|파산\(면책\)")
# p_grt = re.compile(r"개인회생\(면책\)|파산\(면책\)|환매|매각|종결|보증면탈")

def preprocess(file:str) :
    
    df = pd.read_excel(join(wd, file))
    
    for c, v in tqdm(df.items(), total=len(df.columns)) :
        # : 는 공백으로 대체
        try : 
            df[c] = df[c].str.replace(":", " ")
        except :
            print(c)
            break

        # 사건번호에서 공백 제거
        if re.search("사건번호", c) :
            df[c] = df[c].str.replace(" ", "")

    return df

def unite_debt_state (df_debt, df_grt) :
    """df타입의 채무자조회창과, 보증인조회창으로 채무상태만 통합. \n
    return df_united \n
    이후 is_out함수 호출"""
    채무자키 = df_debt["채무자키"]
    채무자상태 = df_debt["채무상태"]
    보증인수 = []
    보증인상태 = []
    for key in tqdm(채무자키, total=len(채무자키), desc="채무상태 통합 중") :
        temp = df_grt[df_grt.채무자키==key]
        보증인수.append(len(temp))
        보증인상태.append(temp.보증인상태.to_list())
    df_united = pd.DataFrame({"채무자키":채무자키, "채무자상태":채무자상태, "보증인수":보증인수, "보증인상태":보증인상태})
    return df_united

def is_out_df(df_united, date_of_data:str) :
    """채무자상태와 보증인상태를 통합한 df를 주면 out칼럼을 추가 \n
    통합상태.xlsx와 outList.pkl을 만듦
    """
    out_list = []
    for i, v in tqdm(df_united.iterrows(), total=len(df_united.index), desc="out여부확인중") :
        if p_totalyOut.match(v.채무자상태) : 
            out_list.append(True)
        elif p_debt.match(v.채무자상태) :  # 채무자는 일단 T(out)
            if v.보증인수 == 0 :
                out_list.append(True)
            else : # 보증인도 모두 out인 경우만 True, 보증인 한명이라도 살아있으면 False -> map과 all()활용
                out_list.append(all(map(lambda x : bool(p_grt.search(x)), v.보증인상태)))
        else : out_list.append(False)
    
    df_united["통합상태"] = out_list

    # 채무자키, 채무자-보증인상태, 통합상태 엑셀저장
    df_united.to_excel(join(wd, "통합상태_"+date_of_data+".xlsx"), index=False)
    # out(True)인경우만 pkl저장( 타입은 sr >> "채무자키" in sr.values )
    outList = df_united[df_united["통합상태"]]["채무자키"].astype(str) # 통합상태는 T/F이므로 바로 불리언 색인가능
    outList.to_pickle(join(wd, "outList_"+date_of_data+".pkl"))


# 채무자키가 주어짐
# def is_out (key:str, df_debt, df_grt) :
#     """채무자조회, 보증인조회에서 단일"키"를 주면 out인지(T) 아닌지(F) 판별해주는 함수"""
#     ### 검색결과가 항상 하나인 시리즈의 경우 values[0] 통해 값을 받을 수 잇다.
#     cond_grt = True
#     # 보증인 무관하게 out 
#     if df_debt[df_debt.채무자키 == key].채무상태.str.match(p_totalyOut).values[0] : #True/False
#         return True
    
#     # 채무자는 out 사용
#     elif df_debt[df_debt.채무자키 == key].채무상태.str.match(p_debt).values[0] :
#         # 보증인들(한명이라도 out이 아니면 False)
#         temp_sr = df_grt[df_grt.채무자키 == key].보증인상태
#         if len(temp_sr) == 0 : # 보증인 없음 따라서 채무자상태에 의해 out
#             return True
#         else :
#             for i in temp_sr :
#                 i = i.strip()
#                 if p_grt.match(i) == None : # 상태 검사 : out상태가 아닌 경우 발생 
#                     cond_grt = False
#                     break # 한명만 out 아니어도 out x
#             return cond_grt
                    
#     else : return False  # 채무자가 out아님

In [ ]:
# ### 전처리, 조회.xlsx > pkl(df타입)
########################################
# file_name_list = ["채무자조회새창_20221220_1128", "보증인새창_20221222_1650", "법조치조회새창_20221220_1246"]
# wd = r"D:\0.전산\2.이관\써머리\전북"
# files = ["채무자조회새창_20230531_1841","계좌조회새창_20230531_1842","보증인새창_20230531_1844", "법조치조회새창_20230531_1843"]
files = ["개인회생새창_20230531_1844"]
########################################
read_ext = ".xlsx"
save_ext = ".pkl"
# 읽기
debt = pd.read_excel(join(wd, files[0]+read_ext), dtype=debt_dtype).fillna("")
account = pd.read_excel(join(wd, files[1]+read_ext), dtype=account_dtype).fillna("")
grt = pd.read_excel(join(wd, files[2]+read_ext), dtype=grt_dtype).fillna("")
law = pd.read_excel(join(wd, files[3]+read_ext), dtype=event_dtype).fillna("")
rehabilitation = pd.read_excel(join(wd, files[4]+read_ext), dtype=rehabilitation_dtype).fillna("")

# 전처리
# 성명에 : 제거
# debt["성명"] = debt["성명"].str.replace(":", " ")
# # 사건번호에서 공백 제거
for c, v in law.items() : 
    if re.search("사건번호", c) :
        law[c] = law[c].str.replace(" ", "")
for c, v in rehabilitation.items() : 
    if re.search("사건번호", c) :
        law[c] = law[c].str.replace(" ", "")

# pkl 저장
debt.to_pickle(join(wd, files[0]+save_ext))
account.to_pickle(join(wd, files[1]+save_ext))
grt.to_pickle(join(wd, files[2]+save_ext))
law.to_pickle(join(wd, files[3]+save_ext))
rehabilitation.to_pickle(join(wd, files[4]+save_ext))

### dict_refer만들기, outList는 이제 빈파일이므로 업데이트 필요없음

In [1]:
# 채무자키,  매각사구분, 채무상태, 채무자성명, 보증인성명 칼럼만 있으면 됨
######################################
wd = r"C:\Users\DATA\Desktop"
fn_debt = "채무자조회새창_20240308_1411"+".xlsx"
######################################

def dict_referFnc(path_df):
    """
    채무자조회새창을 읽어서 아래의 dict를 만들어주는 함수
    dict_refer["key"][0:매각사, 1:채무상태, 2:채무자성명, 3:보증인성명]
    이제 채무상태는 필요없지만 수정하기 귀찮으니 그냥 둔다."""
    df_c = pd.read_excel(path_df, dtype=debt_dtype).fillna("")
    return dict(map(lambda x : (str(x[1].채무자키),[x[1].매각사구분, x[1].채무상태, x[1].성명, x[1].보증인성명]), df_c.iterrows()))

In [4]:
date_of_data = fn_debt.split("_")[1]

dict_refer = dict_referFnc(join(wd, fn_debt)) # 
dict_refer_filename = "dict_refer_"+date_of_data+".pkl"

print(len(dict_refer))

# dict_refer.pkl 만들기
with open(join(wd, dict_refer_filename), 'wb') as pkl :
    pickle.dump(dict_refer, pkl)
    
# load 확인
with open(join(wd, dict_refer_filename), 'rb') as pkl :
    dict_refer1 = pickle.load(pkl)

cnt = 0
for k, v in dict_refer1.items() :
    print(k, v) 
    cnt+=1
    if cnt>3 : break

c:\Users\DATA\AppData\Local\anaconda3\envs\py_39\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


71009
20495151 ['전북-01', '종결', '(유)가람건설', '김원목']
20495152 ['전북-01', '종결', '(유)가람건설', '김원목']
20495238 ['전북-01', '종결', '(유)가람건설', '김원목']
20417854 ['DNP-01', '법인', '(유)가람관광여행사', '김길배']


### 위 까지만 하면 됨. 아래는 이제 안 씀 240308

In [ ]:
# 빈 outList 만들기
df = pd.DataFrame(None, columns=["채무자키"])
df["채무자키"].to_pickle(join(wd, "outList.pkl"))

In [8]:
# outList 만들기
############################################
debt_newwin_pkl = "채무자조회새창_20221220_1128.pkl"
grt_newwin_pkl = "보증인새창_20221220_1143.pkl"
############################################
date_of_data = debt_newwin_pkl.split("_")[1]

df_debt = pd.read_pickle(join(wd, debt_newwin_pkl))
df_grt = pd.read_pickle(join(wd, grt_newwin_pkl)) 

df_united = unite_debt_state(df_debt, df_grt)
is_out_df(df_united, date_of_data)

out여부확인중: 100%|██████████| 70793/70793 [00:01<00:00, 38597.19it/s]


##### outList 예외 반영하기

In [5]:
# ex) 파산면책인데 당사누락인 경우 
# 신경용 20415606, 20431042
#######################
path_outList_pkl = join(wd, "outList_20221220.pkl")
temp = pd.read_pickle(path_outList_pkl)
len(temp)

10232

In [6]:
keys = ["20431042"]
for key in keys :
    temp.drop(temp[temp==key].index, inplace=True)
len(temp)

10231

In [7]:
temp.to_pickle(path_outList_pkl)
# outList.pkl 나스 폴더로 복사 ~~

### 통합법조치(사건번호)로 채무자찾기

In [220]:
# pkl 파일 없으면 아래단에서
##############################################################
# 사건번호 있는 파일 읽기
path_pkl = r"D:\3.자산\전산 dataset\대성\230630"
# 기준데이터 말고 개인회생 원본(중지건까지 사건조회 위해)
files_pkl = ["채무자조회새창_20230630_2335", "보증인새창_20230630_2336", "법조치조회새창_20230630_2336", "개인회생새창_20230630_2337"]
# 채무자조회
debt = None
##############################################################
united_event = pd.DataFrame(None, columns=["채무자키","관할법원", "사건번호"])
for i in range(len(files_pkl)) :
    temp = pd.read_pickle(join(path_pkl, files_pkl[i]+".pkl"))
    # 채무자는 따로 저장
    if i == 0 : debt = temp.copy()
    # 사건번호 관련된 열만 남기기
    selected_columns = temp.columns[temp.columns.str.contains("채무자키|사건번호|법원")]
    temp = temp[selected_columns]
    # 불필요한 열 제거하기
    drop_columns = temp.columns[temp.columns.str.contains("타채무자키|법조치관할법원|법조치사건번호|관할법원코드|청구금액법원|채권자법원")] 
    temp = temp.drop(drop_columns, axis=1)
    # 법원-사건번호 묶음별로 united_event에 concat 하기
    c = 1
    while c < len(temp.columns) :
        added = temp.iloc[:, [0, c, c+1]]
        
        added.columns = united_event.columns
        united_event = pd.concat([united_event, added], ignore_index=True)
        c += 2

In [222]:
# 작업 잘못됐을 때는 대비해 복사
df = united_event.copy()

In [223]:
# 전처리 
p_event = re.compile(r"(\d{2,4})\s?(준?재?[가간나느다머차카타즈본징하개회라정][가-힣]{0,2})\s?(\d+)") # 금제 년제 제외
for i, v in tqdm(df.iterrows(), total=len(df)) :
    res = p_event.search(v.사건번호)

    if res != None :
        year = res[1]

        # 1) 순번 앞에 0이 있으면 지우기
        sn = res[3]
        if re.search("[1-9][0-9]*",res[3]) :
            sn = re.search("[1-9][0-9]*",res[3])[0]

        # 2) 공백 지우기
        event = year + res[2] + sn

        # 3) 연도 수정하기 연도 4자리고, 전후가 동일하면 pass, 수정여부에 x
        if len(year) == 4 :
            df.loc[i,"수정사건번호"] = event
        # 연도 2자리고 00 ~ 23사이면 20 붙이고, 아니면 그대로두기(19는 사건번호에는 안 붙음. 전산조회시 19 있으나 없으나 결과 같음>붙여도 되고 안 붙여도 되고), 수정여부에 ㅇ
        elif len(year) == 2 :
            if int(year) <= 23 :
                df.loc[i,"수정사건번호"] = "20" + event
            elif int(year) > 70  :
                df.loc[i,"수정사건번호"] = event # 원래 19는 안 붙여도 됨. 검색대상에 19가 붙어있는 경우는 제거 하고 검색해야함 # pass 하지말것.
            else : 
                df.loc[i,"수정사건번호"] = "연도이상"
        elif len(year) == 3 :
            if int(year) >= 200 and int(year) <=223 :
                df.loc[i,"수정사건번호"] = event[0] + "0" + event[1:]
            else : 
                df.loc[i,"수정사건번호"] = "연도이상"
        else :
            df.loc[i,"수정사건번호"] = "연도이상"
    else :
        df.loc[i,"수정사건번호"] = "" # 사건번호가 아님

100%|██████████| 47668/47668 [00:03<00:00, 14349.65it/s]


In [224]:
# 사건번호 수정여부 작성하고 바뀐거 체크
df['사건번호수정여부'] = np.where(df['사건번호'] != df['수정사건번호'], 'O', 'X')
df[df['사건번호수정여부'] == 'O']
# 확인없이 바로 저장
df.사건번호 = df.수정사건번호

In [226]:
# 새 df에 저장
df1 = df[["채무자키", "관할법원", "사건번호"]].copy()

# 사건번호 빈값인 거 제거하고, 채무자키와 사건번호가 중복인 거 제거
df1 = df1[df1.사건번호 != ""].drop_duplicates(subset=["채무자키","사건번호"], ignore_index=True)

# 복사위해 새채무자키, 확인차 채무자이름와 주민번호 추가
df2 = debt[["채무자키", "관리자비고", "성명", "주민번호인"]]
df2 = df2.rename(columns={"관리자비고":"새채무자키"})
result = pd.merge(df1, df2, how='left', on='채무자키') 
print(f"merge전 : {len(df1)}개, merge후 : {len(result)}")

merge전 : 12460개, merge후 : 12460


In [227]:
# pkl 저장
path_save = r"D:\1.전산,개발\3.파일서버관리\0.월별전산조회"
filename_save = path_pkl.split('\\')[-2] + "_통합사건번호_" + path_pkl.split('\\')[-1]
result.to_pickle(join(path_save, filename_save+".pkl"))

### 법조치 - 그냥 pkl로만 저장
- preprocess 안 거쳤으면 채무자키 str으로 변환 : df_event.채무자키.apply(str)

In [215]:
wd = r"D:\3.자산\전산 dataset\대성\230630"
filename = "개인회생새창_20230630_2337"

# 엑셀읽기
df_event_ori = pd.read_excel(join(wd, filename + ".xlsx"), dtype=event_dtype).fillna("") # fillna 중요

# pkl 저장

# 필요칼럼만
# df_event = df_event_ori[["채무자키","계좌키","법조치키","대상자구분","대상자명","법조치구분","법조치세부","관할법원","사건번호","사건구분","접수일","확정일","법조치상태","종국결과","청구금액","청구금액법원", "매각사구분", "매각사", "관련사건법원", "관련사건번호", "관련법조치관할법원", "관련법조치사건번호","제소신청일"]].copy()
# df_event.to_pickle(join(r"D:\1.전산,개발\3.파일서버관리\0.월별전산조회", filename+"스캔파일작업용.pkl"))

# 전체저장
df_event = df_event_ori.copy()
df_event.to_pickle(join(wd, filename+".pkl"))

c:\Users\SL\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


### 살릴거, 죽일거

##### 살릴 거(revive = 구out - 신out)

In [40]:
# 신구 outList 읽기
##################################
outList_name_old = "outList_221118.pkl"
outList_name_new = "outList_20221220.pkl"
##################################
stem = os.path.splitext(outList_name_new)[0]
date_of_data = stem.split("_")[1]

set_outList_old = set(pd.read_pickle(join(wd,"2.dict_refer, ouList구버전", outList_name_old)))
set_outList_new = set(pd.read_pickle(join(wd, outList_name_new)))

# 살릴거(revive) 구하기
set_revive = set_outList_old - set_outList_new

# 살릴 거 있으면 저장해두고 작업하자
if len(set_revive) > 0 :
    sr_revive = pd.Series(list(set_revive), name = "살릴채무자키")
    sr_revive.to_excel(join(wd, "revive_"+date_of_data+".xlsx"), index=False)
    print(f"살릴 것이 {len(set_revive)}개 있습니다.")
else : print(f"살릴 것이 없음.")

### 1220 기준 살릴 것 모두 살림

살릴 것이 304개 있습니다.


##### 죽일거(kill = 신out -구out)

In [43]:
# 위 셀 실행 후에 
# 살릴거(revive) 구하기
set_kill = set_outList_new - set_outList_old

# 살릴 거 있으면 저장해두고 작업하자
if len(set_kill) > 0 :
    sr_revive = pd.Series(list(set_kill), name="관리제외할채무자키")
    sr_revive.to_excel(join(wd, "kill_"+date_of_data+".xlsx"), index=False)
    print(f"관리제외할 파일이 서버에 {len(set_kill)}개 있습니다.")
else : print(f"관리제외할 파일이 서버에 없습니다.")

관리제외할 파일이 서버에 439개 있습니다.


##### 신규 종결건 처리하기(죽일거) - nas에서 할 수 있게 만들어야겠다.. 시간 너무 걸려

In [62]:
# 필요 파일 불러오기 ----------------------
# 파일 관리제외할 채무자키 목록(depth3), 매각사 불러올 dict_refer(depth2), 문서구분(depth1)
############################
kill_filename = "kill_20221220.xlsx"
dict_refer_name = "dict_refer_20221220.pkl"
############################
kill = pd.read_excel(join(wd, kill_filename)).iloc[:,0].to_list() # 리스트
dict_refer = pd.read_pickle(join(wd, dict_refer_name)) # dict_refer[key][0] : 매각사구분

docu_folder_dict = {"원인서류": "1.원인서류", "양도통지서": "2.양도통지서", "집행권원": "3.집행권원", "강제집행": "4.강제집행", 
        "등본": "5.등초본", "법인등기" : "5.등초본", "초본": "5.등초본", "등초본":"5.등초본", "외국인증명": "5.등초본", "주민등록정보":"5.등초본", 
        "개인회생": "6.개인회생", "신용회복": "7.신용회복", "파산": "8.파산", "재산조사": "9.재산조사", "부채증명서" : "10.부채증명서", 
        "신용조회" : "11.신용조회", "개인정보":"12.개인정보", "기타": "기타"}
depth1 = list(docu_folder_dict.values())

In [67]:
# dict_kcs 를 위한 import 및 함수
import binascii
def crc32_checksum(filename):
    buf = open(filename,'rb').read()
    buf = (binascii.crc32(buf) & 0xFFFFFFFF)
    return "%08X" % buf

def get_kcs(path_192) :
    crc32 = str(crc32_checksum(path_192))
    size = str(os.path.getsize(path_192))
    f = os.path.split(path_192)[1]
    key = f[:8]
    kcs = key + crc32 + size 
    return kcs

def read_kcs(path_kcs) :
    with open(path_kcs, "rb") as pkl :
        return pickle.load(pkl)

def delete_kcs(filePath, dict_kcs) :
    k = os.path.split(filePath)[1][:8]
    c = str(crc32_checksum(filePath))
    s = str(os.path.getsize(filePath))
    kcs = k + c + s
    if kcs in dict_kcs :
        del dict_kcs[kcs]
        print("kcs를 삭제하였습니다.")
    else :
      print(kcs, "없는 키값")
      
def write_kcs(path_kcs, dict_kcs) :
    with open(path_kcs, "wb") as pkl :
        pickle.dump(dict_kcs, pkl)
    print(len(dict_kcs))

path_kcs = r"\\192.168.0.75\스캔파일\스캔파일log\_project\파일\중복조사\kcs별 파일정보_솔림헬프.pkl"
dict_kcs = read_kcs(path_kcs)
print(len(dict_kcs))

342825


In [ ]:
# (끝) path에 신규 종결건 파일이 있는 경우만 골라서 관리제외 폴더로 옮기기
path = r"\\192.168.0.75\솔림헬프"
out_dir_base = r"\\192.168.0.75\삭제예정파일\관리제외"
path_log = r'\\192.168.0.75\스캔파일\스캔파일log\_project\파일\채무자조회 구버전'
success_key, success, fail = [], [], []
dir_cnt = 0

print(len(kill),"개의 채무자키에 대한 관리제외 작업을 시작합니다.")
print("작업 전 dict_kcs개수 : ", len(dict_kcs))
for d1 in tqdm(depth1, total=13) : 
    for d3 in kill :
        d3 = str(d3)
        d2 = dict_refer[d3][0]
        src_d = join(path, d1, d2, d3)

        if os.path.exists(src_d) :
            dst_d = join(out_dir_base, d2, d3)
            if not os.path.exists(dst_d) :
                os.makedirs(dst_d)

            for f in os.listdir(src_d) :  # 파일 옮기기
                try :
                    # dict_kcs 삭제하기
                    delete_kcs(join(src_d, f), dict_kcs)

                    # 파일 옮기기
                    success.append(re_name_upload(join(src_d, f), join(dst_d, f)))

                except Exception as e :    
                    fail.append([src_d, f, dst_d, f, e.__class__])
                    continue
            
            try : 
                os.rmdir(src_d) # 빈폴더 삭제하기
                dir_cnt += 1
            except Exception as e : 
                print(e)
            success_key.append([d3, "작업완료"])
            print(d3,"에 대한 작업 완료")
        else : 
            print(d3, "폴더, 파일 없어 작업pass")
            success_key.append([d3, "폴더,파일 없음"])

# dict_kcs 저장
write_kcs(path_kcs, dict_kcs)
            
print("작업 후 dict_kcs 개수 : ", len(dict_kcs))
print(len(success),"개의 파일 이동")
print(len(fail),"개의 파일 이동 실패")
print(len(success_key),"개의 dir작업완료")
print(dir_cnt,"개의 폴더 삭제")
write_log_csv_upload(success, path_log, "신규out success")
write_log_csv_upload(fail, path_log, "신규out fail")
write_log_csv_upload(success_key, path_log, "신규out key단위log")

In [109]:
# 전체 os.walk 하는 비효율적인 코드
# path = r"\\192.168.0.75\솔림헬프"
# out_dir_base = r"\\192.168.0.75\관리제외"
# path_log = r'\\192.168.0.75\스캔파일\스캔파일log\_project\파일\채무자조회 구버전'
# success = []
# fail = []
# dir_cnt = 0
# for root, __dirs__, files in tqdm(os.walk(path)) :
#     if re.search("#recycle", root) == None : # 휴지통 nono
#         for key in result :
#             try : 
#                 if re.search(key, root) :
#                     depth2 = root.split(os.path.sep)[-2]
#                     depth3 = root.split(os.path.sep)[-3]
#                     out_dir = join(out_dir_base, depth2, depth3)
                    
#                     # if not os.path.exists(out_dir) :
#                     #     os.makedirs(out_dir)
#                     try :
#                         for f in os.listdir(root) :
#                             success.append([join(root, f), join(out_dir, f)])
#                             # success.append(re_name_upload(join(root, f), join(out_dir, f)))
#                     except Exception as e :    
#                         fail.append([root, f, out_dir, f, e.__class__])
#                         continue
            
#                     # os.rmdir(out_dir)
#                     # dir_cnt += 1
#                     break
#             except Exception as e : 
#                 print(e.with_traceback())
#                 continue
                
# print(len(success),"개의 파일 이동")
# print(len(fail),"개의 파일 이동 실패")
# print(dir_cnt,"개의 폴더 삭제")
# write_log_csv_upload(success, path_log, "success")
# write_log_csv_upload(fail, path_log, "fail")

453 개의 파일 이동
0 개의 파일 이동 실패
0 개의 폴더 삭제
log 파일이 비어있습니다. path : \\192.168.0.75\스캔파일\스캔파일log\_project\파일\채무자조회 구버전


##### 채무자조회 컬럼 추가

In [14]:
import pandas as pd
import pickle

In [16]:
with open(r"D:\0.전산\1.진행중과업\파일서버개편\파일\extraKey\계좌조회새창_20221124_1008.pkl", "rb") as pkl :
    df_left = pickle.load(pkl)
df_left

,채무자키,매각사구분,계좌번호,최초대출일,최초연체일,은행대출원금,최초원금,원채권사,원채권키,채무자명,채무자주민번호,채무상태,타채무자키,타계좌키
0,20412714,DNP-02,1375-16-01001756,2003-04-30,NaN,400000000,400000000,국민은행,0,㈜원숭이학교 대표 정희원,404-81-13967,폐업,0405001212,NaN
1,20412714,DNP-02,5052-16-10081511,2002-06-27,NaN,300000000,250000000,국민은행,0,㈜원숭이학교 대표 정희원,404-81-13967,폐업,0405001212,NaN
2,20412714,DNP-02,5052-16-10081863,2002-07-23,NaN,200000000,197799852,국민은행,0,㈜원숭이학교 대표 정희원,404-81-13967,폐업,0405001212,NaN
3,20412714,DNP-02,5052-16-10082013,2002-07-30,NaN,100000000,100000000,국민은행,0,㈜원숭이학교 대표 정희원,404-81-13967,폐업,0405001212,NaN
4,20455180,DNP-01,8288-09-01017811,2000-10-20,NaN,0,0,국민은행,0,전윤선,541012-2683023,매각,0405001221,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78149,20527985,카카오-01,3650031435424,2018-04-06,2022-04-07,37500000,36765870,카카오뱅크,0,김종연,750529-1390310,신용회복(확정),3650031435424,CCRS-467
78150,20527986,카카오-01,3650011051361,2017-10-09,2019-10-11,5000000,3251178,카카오뱅크,0,정윤미,841003-2652623,신용회복(확정),3650011051361,CCRS-1023
78151,20527987,카카오-01,3650013698082,2018-09-21,2021-06-18,21000000,21000000,카카오뱅크,0,박정업,720215-1221241,개인회생(확정),3650013698082,IRL-846
78152,20527988,카카오-01,3650030453818,2019-06-12,2020-12-10,3000000,22777210,카카오뱅크,0,김태수,890406-1111416,개인회생(확정),3650030453818,IRL-815


In [17]:
# df_right = pd.read_excel(r'c://Users/SL/Desktop/')
# df_right
with open(r"D:\0.전산\1.진행중과업\파일서버개편\파일\extraKey\전화 주소_20221124_1313.pkl", "rb") as pkl :
    df_right = pickle.load(pkl)
df_right

,채무자키,자택전화인,직장전화인,핸드폰인,자택주소,자택번지인,직장주소,직장번지인,초본주소,초본번지인
0,20495151,063-227-0416,NaN,NaN,전라북도 임실군 신평면,석등슬치로 354-1,NaN,NaN,NaN,NaN
1,20495152,063-227-0416,NaN,NaN,전라북도 임실군 신평면,석등슬치로 354-1,NaN,NaN,NaN,NaN
2,20495238,063-227-0416,NaN,NaN,전라북도 임실군 신평면,석등슬치로 354-1,NaN,NaN,NaN,NaN
3,20417854,063-832-5411,063-834-9912,011-944-6480,전북 익산시 어양동,510-1 어양아파트 12-411,전북 익산시 어양동,657-4번지,NaN,NaN
4,20411449,NaN,063-644-3008,010-4652-6956,NaN,NaN,전북 임실군 오수면,금암리 761-3번지,전북 임실군 오수면 금암1길 39-16,(금암리)
...,...,...,...,...,...,...,...,...,...,...
70787,20459682,NaN,051-660-3056,010-9512-0243,부산 남구 용소로64번길 60,"301호 (대연동,쿨하우스)",부산 남구 용당동,산100번지 부경대학교용당캠퍼스 한미르관 4층,NaN,NaN
70788,20459636,NaN,051-205-5177,010-8687-5147,부산 강서구 녹산산업중로 409,(송정동),부산 강서구 녹산산업중로 409,(송정동),NaN,NaN
70789,20447783,NaN,043-530-3200,NaN,서울 용산구 이태원로 223,534호 (한남동),충북 진천군 이월면 생거진천로 1770,(중산리),"전북 익산시 고봉로 146,","7동306호 (마동,주공1차아파트)"
70790,20459338,NaN,NaN,010-7659-9939,서울 용산구 신흥로11나길 17,(용산동2가),0,0,NaN,NaN


In [5]:
# 삭제할 col 있는 경우
# df_right.drop(['은행대출원금', '최초원금'], axis=1, inplace=True)
# df_right

,채무자키,현재원금
0,20455180,0
1,20455180,0
2,20455181,0
3,20411129,0
4,20412715,0
...,...,...
78149,20419638,999109587
78150,20429686,0
78151,20430085,0
78152,20494438,89420225


In [18]:
# 결과가 이상하면 기준셀이 key가 아닌지 확인하라.
df_result = pd.merge(df_left, df_right, how='left')
df_result

,채무자키,매각사구분,계좌번호,최초대출일,최초연체일,은행대출원금,최초원금,원채권사,원채권키,채무자명,...,타계좌키,자택전화인,직장전화인,핸드폰인,자택주소,자택번지인,직장주소,직장번지인,초본주소,초본번지인
0,20412714,DNP-02,1375-16-01001756,2003-04-30,NaN,400000000,400000000,국민은행,0,㈜원숭이학교 대표 정희원,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"전북 부안군 상서면 부안로 1783,",(원숭이학교)
1,20412714,DNP-02,5052-16-10081511,2002-06-27,NaN,300000000,250000000,국민은행,0,㈜원숭이학교 대표 정희원,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"전북 부안군 상서면 부안로 1783,",(원숭이학교)
2,20412714,DNP-02,5052-16-10081863,2002-07-23,NaN,200000000,197799852,국민은행,0,㈜원숭이학교 대표 정희원,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"전북 부안군 상서면 부안로 1783,",(원숭이학교)
3,20412714,DNP-02,5052-16-10082013,2002-07-30,NaN,100000000,100000000,국민은행,0,㈜원숭이학교 대표 정희원,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"전북 부안군 상서면 부안로 1783,",(원숭이학교)
4,20455180,DNP-01,8288-09-01017811,2000-10-20,NaN,0,0,국민은행,0,전윤선,...,NaN,NaN,NaN,NaN,경북 경산시 삼북동,266-37,NaN,NaN,"경북 경주시 건천읍 부산성길 271-28,",(만교사)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78149,20527985,카카오-01,3650031435424,2018-04-06,2022-04-07,37500000,36765870,카카오뱅크,0,김종연,...,CCRS-467,NaN,NaN,010-9138-4420,서울 양천구 목동중앙본로,10 (극동늘푸른아파트) 101-604,NaN,NaN,NaN,NaN
78150,20527986,카카오-01,3650011051361,2017-10-09,2019-10-11,5000000,3251178,카카오뱅크,0,정윤미,...,CCRS-1023,NaN,NaN,010-4748-0609,서울특별시 동대문구 장한로,"111-1, 903호 (장안동, sm타워)",NaN,NaN,NaN,NaN
78151,20527987,카카오-01,3650013698082,2018-09-21,2021-06-18,21000000,21000000,카카오뱅크,0,박정업,...,IRL-846,NaN,NaN,010-6235-2501,충청남도 아산시 음봉면,"음봉로 847, 101-415 (삼일원앙아파트)",NaN,NaN,NaN,NaN
78152,20527988,카카오-01,3650030453818,2019-06-12,2020-12-10,3000000,22777210,카카오뱅크,0,김태수,...,IRL-815,NaN,NaN,010-5125-9927,부산 사상구 광장로86번길,32 (E클레스) 101호,NaN,NaN,NaN,NaN


In [19]:
df_result.to_pickle(r"D:\0.전산\1.진행중과업\파일서버개편\파일\extraKey\계좌_주소_통합본.pkl")